In [ ]:
import os
import respiratory_extraction.utils as utils

data_path = os.path.join(os.getcwd(), '..', 'data', 'subjects')
dataset = utils.Dataset(data_path)

subject = 'Proband05'
scenario = '101_natural_lighting'

video_path = dataset.get_video_path(subject, scenario)

In [ ]:
import argparse

parser = argparse.ArgumentParser('Lightweight Video-based Respiration Rate Detection Algorithm script', add_help=False)
parser.add_argument('--video-path', default='./test2.mp4', help='Video input path')

# # Optical flow parameters
parser.add_argument('--OFP-maxCorners', default=100, type=int, help='')
parser.add_argument('--OFP-qualityLevel', default=0.1, type=float, help='')
parser.add_argument('--OFP-minDistance', default=7, type=int, help='')
parser.add_argument('--OFP-mask', default=None, help='')
parser.add_argument('--OFP-QualityLevelRV', default=0.05, type=float, help='QualityLeve reduction value')
parser.add_argument('--OFP-winSize', default=(15, 15), help='')
parser.add_argument('--OFP-maxLevel', default=2, type=int, help='')

# # FeaturePoint Selection Strategy parameters
parser.add_argument('--FSS-switch', action='store_true', dest='FSS_switch')
parser.add_argument('--FSS-maxCorners', default=100, type=int, help='')
parser.add_argument('--FSS-qualityLevel', default=0.1, type=float, help='')
parser.add_argument('--FSS-minDistance', default=7, type=int, help='')
parser.add_argument('--FSS-mask', default=None, help='')
parser.add_argument('--FSS-QualityLevelRV', default=0.05, type=float, help='QualityLeve reduction value')
parser.add_argument('--FSS-FPN', default=5, type=int,
                    help='The number of feature points for the feature point selection strategy')

# # CCorrelation-Guided Optical Flow Method parameters
parser.add_argument('--CGOF-switch', action='store_true', dest='CGOF_switch')

# # Filter parameters
parser.add_argument('--Filter-switch', action='store_true', dest='Filter_switch')
parser.add_argument('--Filter-type', default='bandpass', help='')
parser.add_argument('--Filter-order', default=3, type=int, help='')
parser.add_argument('--Filter-LowPass', default=2, type=int, help='')
parser.add_argument('--Filter-HighPass', default=40, type=int, help='')

# # Normalization parameters
parser.add_argument('--Normalization-switch', action='store_true', dest='Normalization_switch')

# # RR Evaluation parameters
parser.add_argument('--RR-switch', action='store_true', dest='RR_switch')

# # RR Algorithm parameters
parser.add_argument('--RR-Algorithm-PC-Height', default=None, help='')
parser.add_argument('--RR-Algorithm-PC-Threshold', default=None, help='')
parser.add_argument('--RR-Algorithm-PC-MaxRR', default=45, type=int, help='')
parser.add_argument('--RR-Algorithm-CP-shfit_distance', default=15, type=int, help='')
parser.add_argument('--RR-Algorithm-NFCP-shfit_distance', default=15, type=int, help='')
parser.add_argument('--RR-Algorithm-NFCP-qualityLevel', default=0.6, type=float, help='')

args = parser.parse_args("")

In [ ]:
from scipy.fftpack import fft
from scipy.signal import find_peaks


class RR_Algorithm:
    def __init__(self, data, fs: float):
        self.data = data
        self.fs = fs
        self.N = len(data)
        self.Time = self.N / fs

    # # Fast Fourier Transform Method
    def FFT(self):
        fft_y = fft(self.data)
        maxFrequency = self.fs
        f = np.linspace(0, maxFrequency, self.N)
        abs_y = np.abs(fft_y)
        normalization_y = abs_y / self.N
        normalization_half_y = normalization_y[range(int(self.N / 2))]
        sorted_indices = np.argsort(normalization_half_y)
        RR = f[sorted_indices[-2]] * 60
        return RR

    # # Peak Counting Method
    def PeakCounting(self, Height=args.RR_Algorithm_PC_Height, Threshold=args.RR_Algorithm_PC_Threshold,
                     MaxRR=args.RR_Algorithm_PC_MaxRR):
        Distance = 60 / MaxRR * self.fs
        peaks, _ = find_peaks(self.data, height=Height, threshold=Threshold, distance=Distance)
        RR = len(peaks) / self.Time * 60
        return RR

    # # Crossover Point Method
    def CrossingPoint(self):
        shfit_distance = int(self.fs / 2)
        data_shift = np.zeros(self.data.shape) - 1
        data_shift[shfit_distance:] = self.data[:-shfit_distance]
        cross_curve = self.data - data_shift

        zero_number = 0
        zero_index = []
        for i in range(len(cross_curve) - 1):
            if cross_curve[i] == 0:
                zero_number += 1
                zero_index.append(i)
            else:
                if cross_curve[i] * cross_curve[i + 1] < 0:
                    zero_number += 1
                    zero_index.append(i)

        cw = zero_number
        N = self.N
        fs = self.fs
        RR1 = ((cw / 2) / (N / fs)) * 60

        return RR1

    def NegativeFeedbackCrossoverPointMethod(self, QualityLevel=args.RR_Algorithm_NFCP_qualityLevel):
        shfit_distance = int(self.fs / 2)
        data_shift = np.zeros(self.data.shape) - 1
        data_shift[shfit_distance:] = self.data[:-shfit_distance]
        cross_curve = self.data - data_shift

        zero_number = 0
        zero_index = []
        for i in range(len(cross_curve) - 1):
            if cross_curve[i] == 0:
                zero_number += 1
                zero_index.append(i)
            else:
                if cross_curve[i] * cross_curve[i + 1] < 0:
                    zero_number += 1
                    zero_index.append(i)

        cw = zero_number
        N = self.N
        fs = self.fs
        RR1 = ((cw / 2) / (N / fs)) * 60

        if (len(zero_index) <= 1):
            RR2 = RR1
        else:
            time_span = 60 / RR1 / 2 * fs * QualityLevel
            zero_span = []
            for i in range(len(zero_index) - 1):
                zero_span.append(zero_index[i + 1] - zero_index[i])

            while (min(zero_span) < time_span):
                doubt_point = np.argmin(zero_span)
                zero_index.pop(doubt_point)
                zero_index.pop(doubt_point)
                if len(zero_index) <= 1:
                    break
                zero_span = []
                for i in range(len(zero_index) - 1):
                    zero_span.append(zero_index[i + 1] - zero_index[i])

            zero_number = len(zero_index)
            cw = zero_number
            N = self.N
            fs = self.fs
            RR2 = ((cw / 2) / (N / fs)) * 60

        return RR2

In [ ]:
import cv2
import numpy as np
from scipy import signal


def FeaturePointSelectionStrategy(Image, FPN=5, QualityLevel=0.3):
    Image_gray = Image
    feature_params = dict(maxCorners=args.FSS_maxCorners,
                          qualityLevel=QualityLevel,
                          minDistance=args.FSS_minDistance)

    p0 = cv2.goodFeaturesToTrack(Image_gray, mask=args.FSS_mask, **feature_params)

    """ Robust checking """
    while (p0 is None):
        QualityLevel = QualityLevel - args.FSS_QualityLevelRV
        feature_params = dict(maxCorners=args.FSS_maxCorners,
                              qualityLevel=QualityLevel,
                              minDistance=args.FSS_minDistance)
        p0 = cv2.goodFeaturesToTrack(Image_gray, mask=None, **feature_params)

    if len(p0) < FPN:
        FPN = len(p0)

    h = Image_gray.shape[0] / 2
    w = Image_gray.shape[1] / 2

    p1 = p0.copy()
    p1[:, :, 0] -= w
    p1[:, :, 1] -= h
    p1_1 = np.multiply(p1, p1)
    p1_2 = np.sum(p1_1, 2)
    p1_3 = np.sqrt(p1_2)
    p1_4 = p1_3[:, 0]
    p1_5 = np.argsort(p1_4)

    FPMap = np.zeros((FPN, 1, 2), dtype=np.float32)
    for i in range(FPN):
        FPMap[i, :, :] = p0[p1_5[i], :, :]

    return FPMap


def CorrelationGuidedOpticalFlowMethod(FeatureMtx_Amp, RespCurve):
    CGAmp_Mtx = FeatureMtx_Amp.T
    CGAmpAugmented_Mtx = np.zeros((CGAmp_Mtx.shape[0] + 1, CGAmp_Mtx.shape[1]))
    CGAmpAugmented_Mtx[0, :] = RespCurve
    CGAmpAugmented_Mtx[1:, :] = CGAmp_Mtx

    Correlation_Mtx = np.corrcoef(CGAmpAugmented_Mtx)
    CM_mean = np.mean(abs(Correlation_Mtx[0, 1:]))
    Quality_num = (abs(Correlation_Mtx[0, 1:]) >= CM_mean).sum()
    QualityFeaturePoint_arg = (abs(Correlation_Mtx[0, 1:]) >= CM_mean).argsort()[0 - Quality_num:]

    CGOF_Mtx = np.zeros((FeatureMtx_Amp.shape[0], Quality_num))

    for i in range(Quality_num):
        CGOF_Mtx[:, i] = FeatureMtx_Amp[:, QualityFeaturePoint_arg[i]]

    CGOF_Mtx_RespCurve = np.sum(CGOF_Mtx, 1) / Quality_num

    return CGOF_Mtx_RespCurve


def ImproveOpticalFlow(
        frames: np.ndarray,
        fps: int,
        QualityLevel=0.3,
        FSS=False,
        CGOF=False,
        filter=True,
        Normalization=False,
        RR_Evaluation=False):
    old_frame = frames[0]
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    feature_params = {
        'maxCorners': args.OFP_maxCorners,
        'qualityLevel': QualityLevel,
        'minDistance': args.OFP_minDistance
    }
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=args.OFP_mask, **feature_params)

    """ Robust Checking """
    while p0 is None:
        QualityLevel = QualityLevel - args.OFP_QualityLevelRV
        feature_params = dict(maxCorners=args.OFP_maxCorners,
                              qualityLevel=QualityLevel,
                              minDistance=args.OFP_minDistance)
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    """ FeaturePoint Selection Strategy """
    if FSS:
        p0 = FeaturePointSelectionStrategy(Image=old_gray, FPN=args.FSS_FPN, QualityLevel=args.FSS_qualityLevel)

    else:
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    lk_params = dict(winSize=args.OFP_winSize, maxLevel=args.OFP_maxLevel)
    total_frame = len(frames)

    FeatureMtx = np.zeros((int(total_frame), p0.shape[0], 2))
    FeatureMtx[0, :, 0] = p0[:, 0, 0].T
    FeatureMtx[0, :, 1] = p0[:, 0, 1].T
    frame_num = 1

    while frame_num < total_frame:
        frame_num += 1
        frame = frames[frame_num - 1]
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        pl, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        old_gray = frame_gray.copy()
        p0 = pl.reshape(-1, 1, 2)
        FeatureMtx[frame_num - 1, :, 0] = p0[:, 0, 0].T
        FeatureMtx[frame_num - 1, :, 1] = p0[:, 0, 1].T

    FeatureMtx_Amp = np.sqrt(FeatureMtx[:, :, 0] ** 2 + FeatureMtx[:, :, 1] ** 2)
    RespCurve = np.sum(FeatureMtx_Amp, 1) / p0.shape[0]

    """ CCorrelation-Guided Optical Flow Method """
    if CGOF:
        RespCurve = CorrelationGuidedOpticalFlowMethod(FeatureMtx_Amp, RespCurve)

    fs = fps

    """" Filter """
    if filter:
        original_signal = RespCurve
        #
        filter_order = args.Filter_order
        LowPass = args.Filter_LowPass / 60
        HighPass = args.Filter_HighPass / 60
        b, a = signal.butter(filter_order, [2 * LowPass / fs, 2 * HighPass / fs], args.Filter_type)
        filtedResp = signal.filtfilt(b, a, original_signal)
    else:
        filtedResp = RespCurve

    """ Normalization """
    if Normalization:
        Resp_max = max(filtedResp)
        Resp_min = min(filtedResp)
        Resp_norm = (filtedResp - Resp_min) / (Resp_max - Resp_min) - 0.5
    else:
        Resp_norm = filtedResp

    """ RR Evaluation"""
    RR_method = RR_Algorithm(Resp_norm, fs)
    RR_FFT = RR_method.FFT()
    RR_PC = RR_method.PeakCounting()
    RR_CP = RR_method.CrossingPoint()
    RR_NFCP = RR_method.NegativeFeedbackCrossoverPointMethod()

    if RR_Evaluation:
        return 1 - Resp_norm, round(RR_FFT, 2), round(RR_PC, 2), round(RR_CP, 2), round(RR_NFCP, 2)
    else:
        return 1 - Resp_norm

In [ ]:
import cv2
import matplotlib.pyplot as plt

frames, params = dataset.read_video_bgr(subject, scenario)
video_fs = params.fps

Resp, RR_FFT, RR_PC, RR_CP, RR_NFCP = ImproveOpticalFlow(frames,
                                                         video_fs,
                                                         QualityLevel=args.OFP_qualityLevel,
                                                         FSS=True,
                                                         CGOF=True,
                                                         filter=True,
                                                         Normalization=True,
                                                         RR_Evaluation=True)
print('RR-FFT: {}(bpm)\nRR-PC: {}(bpm)\nRR-CP: {}(bpm)\nRR-NFCP: {}(bpm)'.format(RR_FFT, RR_PC, RR_CP, RR_NFCP))
t = np.linspace(1, len(Resp) / video_fs, len(Resp))

plt.plot(t, Resp)
plt.xlabel("Time ( s )")
plt.title(
    'RR-FFT: {}(bpm)      RR-PC: {}(bpm)\nRR-CP: {}(bpm)      RR-NFCP: {}(bpm)'.format(RR_FFT, RR_PC, RR_CP, RR_NFCP))
plt.show()